In [1]:
from jinja2 import Environment
from jinja2 import FileSystemLoader
from datetime import datetime
import json
import os
import re
from pathlib import Path

In [2]:
import sys
sys.path.append('../utilities')

In [3]:
BASE_PATH = Path('.')
JOB_MARKER_PATH = BASE_PATH / 'jobmarkers'
JOB_PATH = BASE_PATH / 'jobs'
STAGE_PATH = BASE_PATH / 'stages'
BASE_STAGE_PATH = STAGE_PATH / 'base'
from utilities.configs.importsconf import conf as importsconfig
from utilities.configs.stagesconf import conf as stagesconfig

In [4]:
input_file = json.load(open('./temp/inputs.json', 'r'))
input_name = input_file['name']
input_steps = input_file['sequence']
print('input_steps', input_steps)

input_steps [{'id': '1', 'param_out_var': '_df', 'stage': 'sample_stage'}]


In [5]:
datefile = datetime.now().strftime('%Y%m%d_%H-%M-%S')
outputfile = JOB_PATH / '{}.py'.format(input_name)
outputbatfile = JOB_PATH / '{}.bat'.format(input_name)

In [6]:
def render(tpl_path, context):
    path, filename = os.path.split(tpl_path)
    return Environment(loader=FileSystemLoader(path or './')).get_template(filename).render(context)

In [19]:
if os.name == 'nt':
    # on windows
    output = open(str(outputfile), 'w')
    outputbat = open(str(outputbatfile), 'w')
    output.write(open(str(BASE_STAGE_PATH / 'imports.py'), "r").readlines())
else:
    output = open(outputfile.resolve(), 'w')
    outputbat = open(outputbatfile.resolve(), 'w')
    output.write(open((BASE_STAGE_PATH / 'imports.py').resolve(), "r").readlines())

In [8]:
to_import = []
rendered = []
list_job_id = []
list_parent_id = []

In [17]:
if os.name == 'nt':
    # on windows
    rendered.append(open(str(BASE_STAGE_PATH / 'start.py'), "r").read())
else:
    rendered.append(open((BASE_STAGE_PATH / 'start.py').resolve(), "r").read())

In [10]:
for step in input_steps:
    if 'parent' not in step:
        step['parent'] = 'start'
        
    stageconf = stagesconfig[step['stage']]
    ren = render(stageconf['path'], step)
    rendered.append(ren + '\n')
    
    # getting needed imports into list
    to_import = to_import + [i for i in stageconf['dependencies'] if i not in to_import]
    
    # adding the ending
    list_job_id.append(step['id'])
    list_parent_id.append(step['parent'])

In [12]:
# looping through import list and write import lines
for imports in to_import:
    importconf = importsconfig[imports]
    importstr = ''
    if 'importclass' in importconf:
        importstr = 'from {} import {}'.format(importconf['package'], importconf['importclass'])
    else:
        importstr = 'import {}'.format(importconf['package'])
    
    if 'alias' in importconf:
        importstr = '{} as {}'.format(importstr, importconf['alias'])        
    output.write('{}\n'.format(importstr))
    
sysFolder = '{}/{}'.format(JOB_MARKER_PATH, input_name)
output.write("ctx = {{'sysFolder' : '{}'}}\n".format(sysFolder))

48

In [13]:
for item in rendered:
    output.write("{}\n".format(item))

In [14]:
# prep bat file
outputbat.write('python -m luigi --module {} stage_end\n'.format(input_name))

48

In [20]:
outputbat.close()
output.close()